In [ ]:
#hello

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# !cp /content/drive/MyDrive/utils -r ./
# !cp /content/drive/MyDrive/data.zip -r ./

In [6]:
# !rm data.zip

In [14]:
import torch
from utils.dataloader_image_classification import ImageTransform,make_datapath_list,HymenopteraDataset
# 导入软件包
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

In [12]:
#创建保存蚂蚁和蜜蜂的图片文件路径列表
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

#创建dataset
size = 224
mean = (0.485,0.456,0.406)
std = (0.229,0.224,0.225)
train_dataset = HymenopteraDataset(train_list,transform=ImageTransform(size,mean,std),
            phase = "train")
val_dataset = HymenopteraDataset(val_list,ImageTransform(size,mean,std),phase='val')

#创建dataloader
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,batch_size,shuffle=True)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,batch_size,shuffle=False)

#集中保存
dataloader_dict = {"train":train_dataloader, "val":val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [16]:
#创建网络模型

#创建vgg16的实例
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

#将vgg16最后输出层的神经元替换成蚂蚁和蜜蜂
net.classifier[6] = nn.Linear(in_features=4096,out_features=2)
# net

#设置完训练模式
net.train()

print("网络设置完毕：载入已经学习完毕的权重，并设置为训练模式")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


网络设置完毕：载入已经学习完毕的权重，并设置为训练模式


In [17]:
#定义损失函数
criterion = nn.CrossEntropyLoss()

## 设置最优化算法
与迁移学习不同，微调会设置全部网络层参数都可进行学习。


In [24]:
#将微调中需要的学习参数保存到变量params_to_update的1~3中

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

#指定需要学习的网络层名称
update_param_names_1 = ['features']
update_param_names_2 = ['classifier.0.weight','classifier.0.bias',
             'classifier.3.weight','classifier.3.bias']
update_param_names_3 = ['classifier.6.weight','classifier.6.bias']

#将各个参数分别保存到各个例表中
for name,param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print(f"保存到params_to_update_1中: {name}")
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print(f"保存到params_to_update_2中: {name}")
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print(f"保存到params_to_update_3中: {name}")
    else:
        param.requires_grad = False
        print(f"不进行梯度计算: {name}")



保存到params_to_update_1中: features.0.weight
保存到params_to_update_1中: features.0.bias
保存到params_to_update_1中: features.2.weight
保存到params_to_update_1中: features.2.bias
保存到params_to_update_1中: features.5.weight
保存到params_to_update_1中: features.5.bias
保存到params_to_update_1中: features.7.weight
保存到params_to_update_1中: features.7.bias
保存到params_to_update_1中: features.10.weight
保存到params_to_update_1中: features.10.bias
保存到params_to_update_1中: features.12.weight
保存到params_to_update_1中: features.12.bias
保存到params_to_update_1中: features.14.weight
保存到params_to_update_1中: features.14.bias
保存到params_to_update_1中: features.17.weight
保存到params_to_update_1中: features.17.bias
保存到params_to_update_1中: features.19.weight
保存到params_to_update_1中: features.19.bias
保存到params_to_update_1中: features.21.weight
保存到params_to_update_1中: features.21.bias
保存到params_to_update_1中: features.24.weight
保存到params_to_update_1中: features.24.bias
保存到params_to_update_1中: features.26.weight
保存到params_to_update_1中: features.26.bias


In [25]:
#设置各个参数的优化算法
optimizer = optim.SGD([
    {'params':params_to_update_1,"lr":1e-4},
    {'params':params_to_update_2,"lr":5e-4},
    {'params':params_to_update_3,"lr":1e-3}],momentum=0.9)

In [27]:
#学习和验证
#使用GPU

def train_model(net,dataloader_dict,criterion,optimizer,num_epochs):
    #初始化
    #确认GPU
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f'使用设备: {device}')

    #将网络输入GPU
    net.to(device)

    #如果网络达到比较稳定的程度，则开启加速
    torch.backends.cudnn.benchmark = True

    #epoch循环
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("----------")

        #每个epoch中训练与验证循环
        for phase in ['train','val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()

            epoch_loss = 0.0
            epoch_corrects = 0

            #为了对未学习性能验证，省略epoch=0的训练
            if epoch==0 and phase=="train":
                continue

            #加载小批量
            for inputs,labels in tqdm(dataloader_dict[phase]):
                #如果GPU可用，则将数据加载到GPU中
                inputs = inputs.to(device)
                labels = labels.to(device)

                #初始化optimizer
                optimizer.zero_grad()

                #计算forward
                with torch.set_grad_enabled(phase == "train"):
                    outputs = net(inputs)
                    loss = criterion(outputs,labels)
                    _,preds = torch.max(outputs,1)

                    #训练时反向传播
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                    #计算结果
                    epoch_loss += loss.item()*inputs.size(0)
                    epoch_corrects = torch.sum(preds == labels.data)

                #显示每轮的loss和准确率
                epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
                epoch_acc = epoch_corrects.double()/len(dataloader_dict[phase].dataset)

                print(f"{phase} Loss:{epoch_loss:.4f} Acc:{epoch_acc:.4f}")



In [28]:
num_epochs = 2
train_model(net,dataloader_dict,criterion,optimizer,num_epochs )

使用设备: cuda:0
Epoch 1/2
----------


 20%|██        | 1/5 [00:10<00:42, 10.64s/it]

val Loss:0.1244 Acc:0.1699


 40%|████      | 2/5 [00:11<00:13,  4.62s/it]

val Loss:0.1247 Acc:0.1503


...

val Loss:0.0180 Acc:0.2092


 80%|████████  | 4/5 [00:01<00:00,  2.30it/s]

val Loss:0.0496 Acc:0.2026


100%|██████████| 5/5 [00:02<00:00,  2.46it/s]

val Loss:0.0450 Acc:0.1503


In [29]:
# !ls

data  drive  sample_data  utils


In [30]:
#保存和读取训练完毕的网络

#保存PyTorch网络参数
save_path = './weights_fine_tuning.pth'
torch.save(net.state_dict(),save_path)

In [31]:
#载入网络参数
load_path = './weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

<All keys matched successfully>

In [32]:
#在GPU上保存的权重在cpu上读取
load_weights = torch.load(load_path,map_location={'cuda:0':'cpu'})
net.load_state_dict(load_weights)

<All keys matched successfully>

# 以上代码均使用colab T4GPU实现